In [1]:
from time import time
import xarray as xr

def _get_year_month(product, tif):
    fn = tif.split('/')[-1]
    fn = fn.replace(product,'')
    fn = fn.replace('.tif','')
    print(fn)
    return fn

def xr_build_cube_concat_ds(tif_list, product, ur_corner, ll_corner):

    start = time()
    my_da_list =[]
    year_month_list = []
    for tif in tif_list:
        #tiffile = 's3://dev-et-data/' + tif
        tiffile = tif
        print(tiffile)
        da = xr.open_rasterio(tiffile)
        #daSub = da.sel(x=slice(ll_corner[0], ur_corner[0]), y=slice(ur_corner[1], ll_corner[1]))
        daSub = da.sel(x=slice(ll_corner[0], ur_corner[0]), y=slice(ll_corner[1], ur_corner[1]))
        #da = da.squeeze().drop(labels='band')
        #da.name=product
        my_da_list.append(daSub)
        tnow = time()
        elapsed = tnow - start
        print(tif, elapsed)
        year_month_list.append(_get_year_month(product, tif))

    da = xr.concat(my_da_list, dim='band')
    da = da.rename({'band':'year_month'})
    da = da.assign_coords(year_month=year_month_list)
    DS = da.to_dataset(name=product)
    return(DS)


In [2]:
def xr_build_cube_concat_ds_one(tif_list, product, x, y):

    start = time()
    my_da_list =[]
    year_month_list = []
    for tif in tif_list:
        #tiffile = 's3://dev-et-data/' + tif
        tiffile = tif
        print(tiffile)
        da = xr.open_rasterio(tiffile)
        daSub = da.sel(x=x, y=y, method='nearest')
        #da = da.squeeze().drop(labels='band')
        #da.name=product
        my_da_list.append(daSub)
        tnow = time()
        elapsed = tnow - start
        print(tif, elapsed)
        year_month_list.append(_get_year_month(product, tif))

    da = xr.concat(my_da_list, dim='band')
    da = da.rename({'band':'year_month'})
    da = da.assign_coords(year_month=year_month_list)
    DS = da.to_dataset(name=product)
    return(DS)


In [3]:
def create_s3_list_of_days(main_prefix, year, output_product_name):
    output_name = f'{output_product_name}_'
    the_list = []
    for i in range(1,366):
        day = f'{i:03d}'
        file_object = main_prefix  + str(year) + '/' + output_name + str(year) + day + '.tif'
        the_list.append(file_object)
    return the_list

In [4]:
main_bucket_prefix='s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/'
year = '1950'
#output_product_name='dd'
output_product_name='etasw'

In [5]:
tif_list = create_s3_list_of_days(main_bucket_prefix, year, output_product_name)

In [6]:
tif_list

['s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/etasw_1950001.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/etasw_1950002.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/etasw_1950003.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/etasw_1950004.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/etasw_1950005.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/etasw_1950006.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/etasw_1950007.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/etasw_1950008.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/etasw_1950009.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/etasw_1950010.tif',
 's3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/19

In [7]:
ur=(-75, 41.05)
ll=(-75.05,41)
#ll=(-75, 41.1)

In [8]:
#ds = xr_build_cube_concat_ds(tif_list, output_product_name, ur, ll)

In [9]:
#ds

In [10]:
x=-75
y=41
# ds_pix=xr_build_cube_concat_ds_one(tif_list, output_product_name, x, y)

In [11]:
#ds_pix

In [12]:
# my_array = ds_pix['etasw'].values

In [13]:
# my_array

In [14]:
# my_index = ds_pix['year_month'].values

In [15]:
# my_index

In [16]:
import pandas as pd

In [17]:
# df = pd.DataFrame(my_array, columns=['etasw',], index=my_index)

In [18]:
# df

In [19]:
main_bucket_prefix='s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/'
year = '1950'

In [23]:
df_list=[]

for product in ['dd', 'srf', 'etasw', 'netet']:
    x=-75
    y=41
    print("==="*30)
    print("processing product",product)
    tif_list = create_s3_list_of_days(main_bucket_prefix, year, product)
    ds_pix=xr_build_cube_concat_ds_one(tif_list, product, x, y)
    
    my_index = ds_pix['year_month'].values
    my_array = ds_pix[product].values
    df = pd.DataFrame(my_array, columns=[product,], index=my_index)
    df_list.append(df)

processing product dd
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950001.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950001.tif 0.5491964817047119
_1950001
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950002.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950002.tif 0.8425767421722412
_1950002
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950003.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950003.tif 1.236790657043457
_1950003
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950004.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950004.tif 1.578397274017334
_1950004
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950005.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950005.tif 1.871740

s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950042.tif 13.96076226234436
_1950042
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950043.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950043.tif 14.262863874435425
_1950043
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950044.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950044.tif 14.619351387023926
_1950044
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950045.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950045.tif 14.972640991210938
_1950045
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950046.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950046.tif 15.280917644500732
_1950046
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950047.tif
s

s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950084.tif 28.068421363830566
_1950084
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950085.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950085.tif 28.40901017189026
_1950085
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950086.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950086.tif 28.81998896598816
_1950086
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950087.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950087.tif 29.110050439834595
_1950087
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950088.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950088.tif 29.4210205078125
_1950088
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950089.tif
s3:/

s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950126.tif 41.99905276298523
_1950126
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950127.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950127.tif 42.30109906196594
_1950127
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950128.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950128.tif 42.61176300048828
_1950128
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950129.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950129.tif 42.905165910720825
_1950129
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950130.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950130.tif 43.242395639419556
_1950130
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950131.tif
s3:

s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950168.tif 55.7678747177124
_1950168
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950169.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950169.tif 56.133047103881836
_1950169
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950170.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950170.tif 56.45244836807251
_1950170
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950171.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950171.tif 56.75913095474243
_1950171
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950172.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950172.tif 57.097620248794556
_1950172
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/dd_1950173.tif
s3:/

...

s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950171.tif 54.22073554992676
_1950171
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950172.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950172.tif 54.57433843612671
_1950172
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950173.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950173.tif 54.893239974975586
_1950173
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950174.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950174.tif 55.22309350967407
_1950174
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950175.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950175.tif 55.59344553947449
_1950175
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35p

s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950211.tif 67.68937730789185
_1950211
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950212.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950212.tif 67.98266267776489
_1950212
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950213.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950213.tif 68.28744101524353
_1950213
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950214.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950214.tif 68.60223889350891
_1950214
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950215.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950215.tif 68.89769315719604
_1950215
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pc

s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950251.tif 80.57275891304016
_1950251
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950252.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950252.tif 80.8686752319336
_1950252
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950253.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950253.tif 81.1666088104248
_1950253
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950254.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950254.tif 81.46760702133179
_1950254
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950255.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950255.tif 81.76069402694702
_1950255
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/

s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950292.tif 92.98570513725281
_1950292
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950293.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950293.tif 93.2890465259552
_1950293
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950294.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950294.tif 93.65284991264343
_1950294
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950295.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950295.tif 94.34874677658081
_1950295
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950296.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950296.tif 94.64989900588989
_1950296
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct

s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950332.tif 105.41601872444153
_1950332
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950333.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950333.tif 105.69879698753357
_1950333
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950334.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950334.tif 106.01169967651367
_1950334
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950335.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950335.tif 106.30665731430054
_1950335
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950336.tif
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_drb35pct/1950/netet_1950336.tif 106.60748291015625
_1950336
s3://dev-et-data/enduser/DelawareRiverBasin/r_01_29_2021_dr

In [24]:
for dframe in df_list:
    print(dframe)

           dd
_1950001  0.0
_1950002  0.0
_1950003  0.0
_1950004  0.0
_1950005  0.0
...       ...
_1950361  0.0
_1950362  0.0
_1950363  0.0
_1950364  0.0
_1950365  0.0

[365 rows x 1 columns]
          srf
_1950001  0.0
_1950002  0.0
_1950003  0.0
_1950004  0.0
_1950005  0.0
...       ...
_1950361  0.0
_1950362  0.0
_1950363  0.0
_1950364  0.0
_1950365  0.0

[365 rows x 1 columns]
             etasw
_1950001  0.327783
_1950002  0.617009
_1950003  0.350104
_1950004  0.427791
_1950005  0.364538
...            ...
_1950361  0.602801
_1950362  0.509108
_1950363  0.599135
_1950364  0.683448
_1950365  0.819907

[365 rows x 1 columns]
           netet
_1950001  0.0000
_1950002  0.0000
_1950003  0.1411
_1950004  0.1751
_1950005  0.1513
...          ...
_1950361  0.0000
_1950362  0.0000
_1950363  0.0000
_1950364  0.0000
_1950365  0.0000

[365 rows x 1 columns]


In [29]:
olena_df = pd.concat(df_list, axis=1)

In [30]:
olena_df

,dd,srf,etasw,netet
_1950001,0.0,0.0,0.327783,0.0000
_1950002,0.0,0.0,0.617009,0.0000
_1950003,0.0,0.0,0.350104,0.1411
_1950004,0.0,0.0,0.427791,0.1751
_1950005,0.0,0.0,0.364538,0.1513
...,...,...,...,...
_1950361,0.0,0.0,0.602801,0.0000
_1950362,0.0,0.0,0.509108,0.0000
_1950363,0.0,0.0,0.599135,0.0000
_1950364,0.0,0.0,0.683448,0.0000


In [32]:
olena_df['dd'].values

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.36411187e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.62508694e+00, 6.22470198e+00, 0.00000000e+00,
      